In [2]:
import json
import time
import requests
import hashlib
import csv
import pandas as pd
from datetime import datetime


In [588]:
class DomClickApi:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({"X-Service": "true",
                                     "Connection": "Keep-Alive",
                                     "User-Agent": "Android; 12; Google; google_pixel_5; 8.72.0; 8720006; ; NONAUTH"
                                     })

        # init (get cookies)
        self.get("https://api.domclick.ru/core/no-auth-zone/api/v1/ensure_session")
        self.get("https://ipoteka.domclick.ru/mobile/v1/feature_toggles")

    def get(self, url, **kwargs):
        self.__update_headers(url, **kwargs)
        result = self.session.get(url, **kwargs)
        #print(self.session.cookies.get_dict())
        return result

    def __update_headers(self, url, **kwargs):
        url = self.__get_prepared_url(url, **kwargs)
        sault = "ad65f331b02b90d868cbdd660d82aba0"
        timestamp = str(int(datetime.now().timestamp()))
        encoded = (sault + url + timestamp).encode("UTF-8")
        h = hashlib.md5(encoded).hexdigest()
        self.session.headers.update({"Timestamp": timestamp,
                                     "Hash": "v1:" + h,
                                     })

    def __get_prepared_url(self, url, **kwargs):
        p = requests.models.PreparedRequest()
        p.prepare(method="GET", url=url, **kwargs)
        return p.url



In [589]:

def pprint_json(json_str):
    try:
        json_object = json.loads(json_str)
        json_formatted_str = json.dumps(json_object, indent=2, ensure_ascii=False).encode('utf8')
        print(json_formatted_str.decode())
    except:
        pass
        print(json_str)


In [351]:
def get_keys(d):
    if isinstance(d, dict):
        for k, v in d.items():
            yield k
            yield from list(get_keys(v))
    elif isinstance(d, list):
        for o in d:
            yield from list(get_keys(o))

In [693]:
remont = 'design' #without_repair design standard well_done
balcon = [2,]
#rooms = []
type_dome = 'loyout' #flat loyout
type_home = 'panel' #monolith  brick panel block

offers_url = 'https://offers-service.domclick.ru/research/v5/offers/'
count_url = 'https://offers-service.domclick.ru/research/v5/offers/count/'

dca = DomClickApi()
res = dca.get(count_url, params={
    "address": "1d1463ae-c80f-4d19-9331-a1b68a85b553",
    "deal_type": "sale",
    "category": "living",
    "offer_type": ["flat", "layout"],
    #"rooms": rooms,
    'renovation': remont,
    "wall_type": type_home,
    "balconies": balcon
    
    
})
print("RES:", res)
print(res.text)
pprint_json(res.text)

count_obj = json.loads(res.text)
total = count_obj["pagination"]["total"]


RES: <Response [200]>
{"success":true,"pagination":{"limit":30,"offset":0,"total":56},"result":{}}
{
  "success": true,
  "pagination": {
    "limit": 30,
    "offset": 0,
    "total": 56
  },
  "result": {}
}


In [694]:
offset = 0
df_data = []
while offset < total:
    res = dca.get(offers_url, params={
        "address": "1d1463ae-c80f-4d19-9331-a1b68a85b553",
        "deal_type": "sale",
        "category": "living",
        "offer_type": ["flat", "layout"],
        "offset": offset,
        "limit": 10,
        #"rooms": rooms,
        "wall_type": type_home,
        "renovation": remont,
        'balconies': balcon
    })
    #print("RES:", res)
    #pprint_json(res.text)
    offset += 10
    offers_obj = json.loads(res.text)
    #print(list(get_keys(offers_obj)))
    total = offers_obj["pagination"]["total"]
    result_data = offers_obj['result']['items']
    for item in result_data:
        address = item['address']
        price = item['price_info']
        house = item['house']
        object_info = item['object_info']

        #renovation = item['renovation']
        row = {
                #'address_id': address['id'],
                'address_name': address['name'],
                'price': price['price'],
                'square_price': price['square_price'],

                'floors': house['floors'],
                #'chat_avail': item['chat_available'],
                
                'rooms': object_info['rooms'],
                'area': object_info['area'],
                'local': address['locality']['name'],
                'type_house': type_home,
                'type_home': type_dome,
                'remont': remont,
                'total_floors': house['floors'],
                'balcon': balcon[0]

        }
            # Находим seo_uri для данной станции метро
            #for seo_subway in seo_info['subways']:
            #    if seo_subway['guid'] == subway['guid']:
            #        row['seo_uri'] = seo_subway['seo_uri']
            #        break

            # Добавляем словарь в список данных
        df_data.append(row)

# Создаем DataFrame
    df = pd.DataFrame(df_data)
    #df = df.drop_duplicates(subset=['address_name'])


In [695]:
df

,address_name,price,square_price,floors,rooms,area,local,type_house,type_home,remont,total_floors,balcon
0,"улица Грекова, 3к2",22500000,350467,16,3,64.20,Москва,panel,loyout,design,16,2
1,"Береговой проезд, 9к5",17600000,451282,12,1,39.00,Москва,panel,loyout,design,12,2
2,"Сивашская улица, 7",36500000,325892,17,4,112.00,Москва,panel,loyout,design,17,2
3,"Окская улица, 5к1",44999999,409090,23,4,110.00,Москва,panel,loyout,design,23,2
4,"Дубнинская улица, 2к1",16499000,260812,12,3,63.26,Москва,panel,loyout,design,12,2
5,"микрорайон Северное Чертаново, 6к605",22900000,394827,16,2,58.00,Москва,panel,loyout,design,16,2
6,"13-я Парковая улица, 42",26000000,342555,14,3,75.90,Москва,panel,loyout,design,14,2
7,"улица Академика Глушко, 14к1",12000000,307692,12,1,39.00,Москва,panel,loyout,design,12,2
8,"улица Липчанского, 6",15600000,242612,17,2,64.30,Москва,panel,loyout,design,17,2
9,"Ленинский проспект, 144к5",19700000,315705,16,2,62.40,Москва,panel,loyout,design,16,2


In [5]:
main = pd.read_csv('data/1_data.csv')
#main = pd.concat([df, main], ignore_index=True)
#main.to_csv('1_data.csv')

In [737]:
#main = main.drop_duplicates(subset=['address_name'])

In [14]:
main = main.drop(['Unnamed: 0.58'], axis = 1, )

In [16]:

main.to_csv('1_data.csv')


In [17]:
main

,address_name,price,square_price,floors,rooms,area,local,type_house,type_home,remont,total_floors,balcon
0,"улица Грекова, 3к2",22500000,350467,16,3,64.20,Москва,panel,loyout,design,16,2
1,"Береговой проезд, 9к5",17600000,451282,12,1,39.00,Москва,panel,loyout,design,12,2
2,"Сивашская улица, 7",36500000,325892,17,4,112.00,Москва,panel,loyout,design,17,2
3,"Окская улица, 5к1",44999999,409090,23,4,110.00,Москва,panel,loyout,design,23,2
4,"Дубнинская улица, 2к1",16499000,260812,12,3,63.26,Москва,panel,loyout,design,12,2
...,...,...,...,...,...,...,...,...,...,...,...,...
35470,"Детский проезд, 1к1",6400000,118518,19,2,54.00,Казань,monolith,flat,standard,19,1
35471,"Нижняя улица, 8",7800000,95823,17,3,81.40,Казань,monolith,flat,standard,17,1
35472,"улица Рауиса Гареева, 109",5650000,200996,23,0,28.11,Казань,monolith,flat,standard,23,1
35473,"улица Нурихана Фаттаха, 17/4",6000000,92307,18,2,65.00,Казань,monolith,flat,standard,18,1
